In [3]:
from subprocess import call 
from glob import glob 
import pandas 

well_map = 'kunkelWellCSV_r18bf4kyjjtv8.csv'

# merge and align the sequences 
call(['bash', 'merge.bash', well_map ])
call(['bash', 'align.bash' ])

df = pandas.read_csv( well_map )

for fil in glob( 'out/*diff' ):
  with open( fil ) as fn:
    sub, que = fn.readlines()[0].split( '\t' )   
    d = '+'.join( [ '{}{}{}'.format(i, n+1, j).lower() for (n, (i, j)) in enumerate( zip( sub, que ) ) if i != j ] ) 
    if fil.split( '/' )[1].split( '.' )[0].split( '-' )[0] == d:
      print fil.split( '/' )[1].split( '.' )[0], 'has the correct sequence' , 
      print 'in well {}'.format( df[( df['mutant name'] == fil.split( '/' )[1].split( '.' )[0] )].well.iget(0) ) 

a227w-3 has the correct sequence in well C4
a236e-1 has the correct sequence in well D1
a249e-2 has the correct sequence in well E7
a249e-3 has the correct sequence in well F7
d121f-3 has the correct sequence in well D5
e281h-1 has the correct sequence in well B8
e281h-2 has the correct sequence in well C8
e353f-1 has the correct sequence in well H5
f343s-1 has the correct sequence in well F6
g12n-2 has the correct sequence in well B1
g12n-3 has the correct sequence in well C1
g153a-1 has the correct sequence in well E5
g153a-2 has the correct sequence in well F5
h315r-1 has the correct sequence in well A7
h315r-3 has the correct sequence in well C7
n163s-1 has the correct sequence in well B2
n163s-3 has the correct sequence in well D2
q19p-1 has the correct sequence in well H2
q19p-3 has the correct sequence in well B3
s32l-1 has the correct sequence in well C3
v320q-1 has the correct sequence in well F3
v320q-2 has the correct sequence in well G3
w407k-2 has the correct sequence in w